# Analysis of synchronized freezing
# How to analye FreezeFrame video (Win10)
    2019/12/26 Bug fix: we need to install win64 ffmpeg
---
## Step 1. Export AVI file using FreezeFrame Viewer
    1) install FreezeFrameInstaller4104.zip
    2) in FreezeFrame Viewer
        Video/Compress Image Files, as AVI file using MJPEG Compressor
---
## Step 2. Conversion avi to mp4
### Rotate and compress all avi videos in the current directory
    Provide current directory

In [4]:
# Ubuntu path
#path = '/home/alexei/ubg-tera05/WD_Passport/Alexei/synchrony/110919'
# Windows path
path = r'Z:\wataru\WD_Passport\Alexei\synchrony\2019_9_12\122419'
path = r'Z:\wataru\WD_Passport\Alexei\synchrony\2019_9_12\122419_wi'
print(path)

Z:\wataru\WD_Passport\Alexei\synchrony\2019_9_12\122419_wi


In [5]:
import os
import ffmpeg

#path = os.getcwd()
#print(path)

os.chdir(path)
print(os.listdir(path))

for file in os.listdir(path):
    base = os.path.splitext(file)[0]
    extn = os.path.splitext(file)[1]
    if extn == '.avi':
        # !ffmpeg -i {file} -vf "transpose=2,transpose=2" -vcodec libx264 {base}.mp4
        fileIN = os.path.join(path,file)
        print('file = ', fileIN, end = ' ')
        stream = ffmpeg.input(fileIN)
        stream = ffmpeg.hflip(stream)
        stream = ffmpeg.vflip(stream)
        fileOUT = os.path.join(path,base) + '.mp4'
        print('---> output = ', fileOUT)
        stream = ffmpeg.output(stream, fileOUT, vcodec='libx264')
        ffmpeg.run(stream)

print('end')

['122419.ffdd', '122419.fftt', 'Freeze_Log.xls', 'm63.avi', 'm63.ffii', 'm64.avi', 'm64.ffii', 'm65.avi', 'm65.ffii', 'm66.avi', 'm66.ffii', 'm67.avi', 'm67.ffii', 'm68s.avi', 'm68s.ffii', 'm69s.avi', 'm69s.ffii', 'm70s.avi', 'm70s.ffii']
file =  Z:\wataru\WD_Passport\Alexei\synchrony\2019_9_12\122419_wi\m63.avi ---> output =  Z:\wataru\WD_Passport\Alexei\synchrony\2019_9_12\122419_wi\m63.mp4
file =  Z:\wataru\WD_Passport\Alexei\synchrony\2019_9_12\122419_wi\m64.avi ---> output =  Z:\wataru\WD_Passport\Alexei\synchrony\2019_9_12\122419_wi\m64.mp4
file =  Z:\wataru\WD_Passport\Alexei\synchrony\2019_9_12\122419_wi\m65.avi ---> output =  Z:\wataru\WD_Passport\Alexei\synchrony\2019_9_12\122419_wi\m65.mp4
file =  Z:\wataru\WD_Passport\Alexei\synchrony\2019_9_12\122419_wi\m66.avi ---> output =  Z:\wataru\WD_Passport\Alexei\synchrony\2019_9_12\122419_wi\m66.mp4
file =  Z:\wataru\WD_Passport\Alexei\synchrony\2019_9_12\122419_wi\m67.avi ---> output =  Z:\wataru\WD_Passport\Alexei\synchrony\2019

import ffmpeg
stream = ffmpeg.input('input.mp4')
stream = ffmpeg.hflip(stream)
stream = ffmpeg.output(stream, 'output.mp4')
ffmpeg.run(stream)

## Step 3. Video Analysis using VideoPlayer
### maximus009/VideoPlayer
    https://github.com/maximus009/VideoPlayer<BR>
    2019/01/19 Fix some bugs
    see VideoPlayer.ipynb

In [6]:
import os, sys, time
import cv2, numpy as np

# Full path of the video file
#video = r'Z:\wataru\WD_Passport\Wataru\082216\m24a.mp4'
path = r'Z:\wataru\WD_Passport\Wataru\082216'
video = 'm24a.mp4'
video = os.path.join(path,video)


def flick(x):
    pass

def process(im):
    return cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

def add_text(img, text, text_top, image_scale):
    """
    Args:
        img (numpy array of shape (width, height, 3): input image
        text (str): text to add to image
        text_top (int): location of top text to add
        image_scale (float): image resize scale

    Summary:
        Add display text to a frame.

    Returns:
        Next available location of top text (allows for chaining this function)
    """
    cv2.putText(
        img=img,
        text=text,
        org=(0, text_top),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=image_scale,
        color=(255, 255, 255))
    return text_top + int(5 * image_scale) 


# Initialize the display windows
cv2.namedWindow('image')
cv2.moveWindow('image',250,150)
cv2.namedWindow('controls')
cv2.moveWindow('controls',250,50)

# Generate image 'controls' with the size of 50 x 750.
controls = np.zeros((50,750),np.uint8)
# Draw text on the image 'controls'
cv2.putText(controls, "W/w: Play, S/s: Stop, A/a: Prev_Frame, D/d: Next_Frame, E/e: Faster, Q/q: Slower, Esc: Exit",\
            (40,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255)



# Open the video file
cap = cv2.VideoCapture(video)

# Get the total number of frame
tots = cap.get(cv2.CAP_PROP_FRAME_COUNT)

i = 0
cv2.createTrackbar('S','image', 0, int(tots)-1, flick)
cv2.setTrackbarPos('S','image',0)

cv2.createTrackbar('F','image', 1, 100, flick)
frame_rate = 30
cv2.setTrackbarPos('F','image',frame_rate)


status_list = { ord('s'):'stay', ord('S'):'stay',
                ord('w'):'play', ord('W'):'play',
                ord('a'):'prev_frame', ord('A'):'prev_frame',
                ord('d'):'next_frame', ord('D'):'next_frame',
                ord('q'):'slow', ord('Q'):'slow',
                ord('e'):'fast', ord('E'):'fast',
                ord('c'):'snap', ord('C'):'snap',
                -1:'no_key_press', 
                27:'exit'}

status = 'stay'
start_time = time.time()
realFrameRate = frame_rate

# The loop
while True:
    cv2.imshow("controls",controls)
    try:

        # If reach to the end, play from the begining
        if i==tots-1:
            i=0
        
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, im = cap.read()

        # put the current state in the image
        im_text = status + " at " + str(realFrameRate) + " fps"
        add_text(im, im_text, 20, 0.5)
        
        #add_text(im, status, 20, 0.5)
        #add_text(im, str(realFrameRate), 40, 0.5)        
        
        r = 750.0 / im.shape[1]
        dim = (750, int(im.shape[0] * r))
        im = cv2.resize(im, dim, interpolation = cv2.INTER_AREA)
        if im.shape[0]>600:
            im = cv2.resize(im, (500,500))
            controls = cv2.resize(controls, (im.shape[1],25))

        cv2.imshow('image', im)

        
        # Read key input
        status_new = status_list[cv2.waitKey(1)]
        if status_new != 'no_key_press':
            status = status_new

        if status == 'play':
            frame_rate = cv2.getTrackbarPos('F','image')
            if frame_rate == 0.0:
                continue        
            if (time.time() - start_time) > 1.0/frame_rate:
                realFrameRate = round(1.0/(time.time() - start_time),2)
                i+=1
                cv2.setTrackbarPos('S','image',i)
                start_time = time.time()
                continue
        elif status == 'stay':
            i = cv2.getTrackbarPos('S','image')
        elif status == 'exit':
            break
        elif status=='prev_frame':
            i-=1
            cv2.setTrackbarPos('S','image',i)
            status='stay'
        elif status=='next_frame':
            i+=1
            cv2.setTrackbarPos('S','image',i)
            status='stay'
        elif status=='slow':
            frame_rate = max(frame_rate - 1, 0)
            cv2.setTrackbarPos('F', 'image', frame_rate)
            status='play'
        elif status=='fast':
            frame_rate = min(100,frame_rate+1)
            cv2.setTrackbarPos('F', 'image', frame_rate)
            status='play'
        elif status=='snap':
            cv2.imwrite("./"+"Snap_"+str(i)+".jpg",im)
            print("Snap of Frame",i,"Taken!")
            status='stay'

    except KeyError:
        print("Invalid Key was pressed")

# Clean up windows
cap.release()
cv2.destroyAllWindows()

Invalid Key was pressed
Invalid Key was pressed


In [2]:
import tkinter as tk

def cbc(id, tex):
    return lambda : callback(id, tex)

def callback(id, tex):
    s = 'At {} f is {}\n'.format(id, id**id/0.987)
    tex.insert(tk.END, s)
    tex.see(tk.END)             # Scroll if necessary

top = tk.Tk()
tex = tk.Text(master=top)
tex.pack(side=tk.RIGHT)
bop = tk.Frame()
bop.pack(side=tk.LEFT)
for k in range(1,10):
    tv = 'Say {}'.format(k)
    b = tk.Button(bop, text=tv, command=cbc(k, tex))
    b.pack()

tk.Button(bop, text='Exit', command=top.destroy).pack()
top.mainloop()